<a href="https://colab.research.google.com/github/DeborahLeite/metabacording_class/blob/main/Workflow_QIIME2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando QIIME 2

1. Baixar e instalar o Miniconda, o conda ambiente e gerenciador de pacotes recomendado pra instalação do QIIME2

In [ ]:
https://conda.io/projects/conda/en/latest/user-guide/install/download.html#

2. Depois de instalar o Miniconda e abrir um novo terminal, verifique se você está executando a versão mais recente de conda:

In [ ]:
conda update conda

3. Instalar wget para baixar o qiime2

In [ ]:
conda install wget

4. Instalar o QIIME2 em ambiente conda (MacOS e Linux)

In [ ]:
wget https://data.qiime2.org/distro/core/qiime2-2023.2-py38-osx-conda.yml
conda env create -n qiime2-2023.2 --file qiime2-2023.2-py38-osx-conda.yml

5. Ative o ambiente conda

In [ ]:
conda activate qiime2-2023.2

6. Teste sua instalação

In [ ]:
qiime --help

# Download de classificadores taxonômicos:

Consulte: https://docs.qiime2.org/2023.2/data-resources/#data-resources

# Importando dados

In [ ]:
cd Documents/Example_Dataset

In [ ]:
qiime tools import  \
--type 'SampleData[PairedEndSequencesWithQuality]'  \
--input-path manifestfile.csv \
--output-path demux-paired-end.qza \
--input-format PairedEndFastqManifestPhred33


In [ ]:
qiime demux summarize \
--i-data demux-paired-end.qza \
--o-visualization demux-paired-end.qzv

# Controle de qualidade das sequências, *Pair joining* e *Clustering*

**Usando DADA2**

In [ ]:
mkdir ASVS_DADA2

In [ ]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs demux-paired-end.qza \
--o-table ASVS_DADA2/asv_table.qza \
--o-representative-sequences ASVS_DADA2/rep-seqs.qza  \
--o-denoising-stats ASVS/denoising-stats.qza \
--p-trim-left-f 15 \
--p-trim-left-r 15 \
--p-trunc-len-f 145 \
--p-trunc-len-r 145

In [ ]:
cp metadatafile.tsv ASVS_DADA2/metadatafile.tsv
cd ASVS_DADA2

Visualização dos dados

In [ ]:
qiime feature-table summarize \
  --i-table asv_table.qza \
  --o-visualization asv_table.qzv \
  --m-sample-metadata-file metadatafile.tsv
 
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv
 
qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

**Usando Deblur**

In [ ]:
mkdir ASVS_DEBLUR

In [ ]:
qiime quality-filter q-score \
--i-demux demux-paired-end.qza \
--o-filtered-sequences ASVS_DEBLUR/demux-q30-filtered.qza \
--o-filter-stats ASVS_DEBLUR/demux-q30-filter-stats.qza \
--p-min-quality 30
 
qiime demux summarize \
--i-data ASVS_DEBLUR/demux-q30-filtered.qza  \
--o-visualization ASVS_DEBLUR/demux-q30-filtered.qzv
 
 
qiime metadata tabulate \
--m-input-file ASVS_DEBLUR/demux-q30-filter-stats.qza \
--o-visualization ASVS_DEBLUR/demux-q30-filter-stats.qzv

In [ ]:
cp metadatafile.tsv ASVS_DEBLUR/metadatafile.tsv
cd ASVS_DEBLUR

In [ ]:
qiime deblur denoise-16S \
--i-demultiplexed-seqs demux-q30-filtered.qza \
--o-representative-sequences rep-seqs.qza \
--o-table asv_table.qza \
--p-sample-stats \
--o-stats deblur-stats.qza \
--p-trim-length 140
 
qiime feature-table tabulate-seqs \
--i-data rep-seqs.qza \
--o-visualization rep-seqs.qzv
 
qiime deblur visualize-stats \
--i-deblur-stats deblur-stats.qza \
--o-visualization deblur-stats.qzv
 
qiime feature-table summarize \
--i-table asv_table.qza \
--o-visualization asv_table.qzv \
--m-sample-metadata-file metadatafile.tsv

# Construção de árvore filogenética

A partir daqui foram usados apenas os arquivos de saída do DADA2

In [ ]:
qiime alignment mafft \
--i-sequences rep-seqs.qza \
--o-alignment aligned-rep-seqs.qza
 
qiime alignment mask \
--i-alignment aligned-rep-seqs.qza \
--o-masked-alignment masked-aligned-rep-seqs.qza
 
qiime phylogeny fasttree \
--i-alignment masked-aligned-rep-seqs.qza \
--o-tree unrooted-tree.qza
 
qiime phylogeny midpoint-root \
--i-tree unrooted-tree.qza \
--o-rooted-tree rooted-tree.qza

# Remoção de sequências de Archaea, mitocôndrias e cloroplastos

In [ ]:
qiime taxa filter-table  \
--i-table asv_table.qza  \
--i-taxonomy taxonomy.qza  \
--p-exclude mitochondria,chloroplast,archaea  \
--o-filtered-table asv-table-woAMC.qza

qiime feature-table summarize \
  --i-table asv-table-woAMC.qza \
  --o-visualization asv-table-woAMC.qzv 

# Identificação taxonômica

Tempo estimado 36 min (DADA2), 16 min (Deblur).

In [ ]:
qiime feature-classifier classify-sklearn \
--i-classifier silva-138-99-nb-classifier.qza \
--i-reads rep-seqs.qza \
--o-classification taxonomy.qza
 
qiime metadata tabulate \
--m-input-file taxonomy.qza \
--o-visualization taxonomy.qzv

# Unir ASV table e taxonomy table

1. Tabela de ASV filtrada (sem Archaea, mitocôndrias e cloroplastos) para fazer a rarefação no R.

In [ ]:
qiime taxa collapse \
--i-table asv-table-woAMC.qza \
--i-taxonomy taxonomy.qza \
--o-collapsed-table collapsed-asv-table-woAMC.qza \
--p-level 7

qiime metadata tabulate  \
--m-input-file collapsed-asv-table-woAMC.qza \
--o-visualization collapsed-asv-table-woAMC.qzv

qiime tools extract  \
--input-path collapsed-asv-table-woAMC.qza  \
--output-path feature-collapsed-asv-table-woAMC


1.1. Converter tabela para formato BIOM e depois TSV

* O código *d24fc059-784a-4d3c-8c8b-6746135f614b* irá variar, por isso use TAB.

In [ ]:
mv feature-collapsed-asv-table-woAMC/d24fc059-784a-4d3c-8c8b-6746135f614b/data/feature-table.biom \
feature-collapsed-asv-table-woAMC/feature-table.biom
 
biom convert -i feature-collapsed-asv-table-woAMC/feature-table.biom \
-o feature-collapsed-asv-table-woAMC/feature-table.tsv \
--to-tsv \
--header-key taxonomy

2. Tabela de ASV rarificada usando **qiime diversity core-metrics-phylogenetic**

In [ ]:
qiime taxa collapse \
--i-table core-metrics-results/rarefied_table.qza \
--i-taxonomy taxonomy.qza \
--o-collapsed-table core-metrics-results/collapsed_table.qza \
--p-level 7
 
qiime metadata tabulate  \
--m-input-file core-metrics-results/collapsed_table.qza  \
--o-visualization core-metrics-results/collapsed_table.qzv

2.1. Converter tabela para formato BIOM e depois TSV

* O código *5b8a8623-f743-45c9-8d56-5cdc6aefcaeb* irá variar, por isso use TAB.

In [ ]:
qiime tools extract  \
--input-path core-metrics-results/collapsed_table.qza  \
--output-path feature-collapsed-table

mv feature-collapsed-table/5b8a8623-f743-45c9-8d56-5cdc6aefcaeb/data/feature-table.biom feature-collapsed-table\feature-table.biom

biom convert  \
-i feature-collapsed-table/feature-table.biom  \
-o feature-collapsed-table/feature-table.tsv  \
--to-tsv  \
--header-key taxonomy

# Beta e Alfa diversidade (Análises preliminares)

--p-sampling-depth de 21300 (DADA2), 11500 (Deblur)

In [ ]:
qiime diversity core-metrics-phylogenetic  \
--i-phylogeny rooted-tree.qza  \
--i-table asv-table-woAMC.qza  \
--p-sampling-depth 21300  \
--output-dir core-metrics-results_naive  \
--m-metadata-file metadatafile.tsv

Convertendo dados de alfadiversidade em arquivos qzv.

In [ ]:
cp metadatafile.tsv core-metrics-results/metadatafile.tsv
cd core-metrics-results

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity shannon_vector.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization shannon-significance.qzv
 
qiime diversity alpha-correlation \
--i-alpha-diversity shannon_vector.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization shannon-significance-association.qzv
 
qiime diversity alpha-group-significance \
--i-alpha-diversity observed_features_vector.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization observed_features-significance.qzv
 
qiime diversity alpha-correlation \
--i-alpha-diversity observed_features_vector.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization observed-features-significance-association.qzv
 
qiime diversity alpha-group-significance \
--i-alpha-diversity faith_pd_vector.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization faith_pd_vector-significance.qzv
 
qiime diversity alpha-correlation \
--i-alpha-diversity faith_pd_vector.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization faith_pd_vector-significance-association.qzv

Como fazer BARPLOTS

In [ ]:
cp taxonomy.qza core-metrics-results/taxonomy.qza
cd core-metrics-results

qiime taxa barplot \
--i-table rarefied_table.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file metadatafile.tsv \
--o-visualization barplot.qzv